# CMSC 320 Term Project

## Data Preprocessing

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import stats
import kagglehub
import os

In [59]:
path1 = kagglehub.dataset_download("mczielinski/bitcoin-historical-data")
path1 = os.path.join(path1, "btcusd_1-min_data.csv")
path2 = kagglehub.dataset_download("odins0n/top-50-cryptocurrency-historical-prices")
path2 = os.path.join(path2, "All_combined.csv")

print("Path to dataset file for Bitcoin Historical Data:", path1)
print("Path to dataset files for Top 50 cryptos historical prices:", path2)

Path to dataset file for Bitcoin Historical Data: C:\Users\rishi\.cache\kagglehub\datasets\mczielinski\bitcoin-historical-data\versions\401\btcusd_1-min_data.csv
Path to dataset files for Top 50 cryptos historical prices: C:\Users\rishi\.cache\kagglehub\datasets\odins0n\top-50-cryptocurrency-historical-prices\versions\5\All_combined.csv


First, we read the csv files, then convert the Timestamp column into Pandas datetime objects, so that we can later use time based operations cleaner. Then we sort the Timestamp column chronologically so we are able to see trends clearer. Finally we drop any duplicated timestamps so that only the first occurence shows, which resolves any skew in the data.

In [60]:
df1 = pd.read_csv(path1)

df1["Timestamp"] = pd.to_datetime(df1["Timestamp"], unit="s")
df1 = df1.set_index("Timestamp").sort_index()
df1 = df1[~df1.index.duplicated(keep="first")]

display(df1.head())

Open  High   Low  Close  Volume
Timestamp                                           
2012-01-01 10:01:00  4.58  4.58  4.58   4.58     0.0
2012-01-01 10:02:00  4.58  4.58  4.58   4.58     0.0
2012-01-01 10:03:00  4.58  4.58  4.58   4.58     0.0
2012-01-01 10:04:00  4.58  4.58  4.58   4.58     0.0
2012-01-01 10:05:00  4.58  4.58  4.58   4.58     0.0

In [61]:
display(df1.tail())

Open      High       Low     Close    Volume
Timestamp                                                            
2025-10-27 23:53:00  114129.0  114182.0  114129.0  114182.0  0.116283
2025-10-27 23:54:00  114186.0  114186.0  114171.0  114171.0  0.037052
2025-10-27 23:55:00  114187.0  114187.0  114187.0  114187.0  0.000045
2025-10-27 23:56:00  114189.0  114189.0  114119.0  114119.0  0.161391
2025-10-27 23:57:00  114105.0  114124.0  114105.0  114124.0  0.079630

Next, we run to see if there are missing values in the data. There appears to be none.

In [62]:
display(df1.isna().sum())

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

Describe this dataset: Gives overview of each column of the dataframe.

In [63]:
display(df1[["Open", "High", "Low", "Close", "Volume"]].describe())

Open          High           Low         Close        Volume
count  7.268797e+06  7.268797e+06  7.268797e+06  7.268797e+06  7.268797e+06
mean   2.073104e+04  2.073850e+04  2.072338e+04  2.073105e+04  5.149885e+00
std    2.933537e+04  2.934328e+04  2.932733e+04  2.933537e+04  2.213713e+01
min    3.800000e+00  3.800000e+00  3.800000e+00  3.800000e+00  0.000000e+00
25%    4.403300e+02  4.405500e+02  4.400800e+02  4.403300e+02  1.904482e-02
50%    7.191870e+03  7.195650e+03  7.187920e+03  7.191760e+03  4.465496e-01
75%    3.022037e+04  3.022755e+04  3.021300e+04  3.022085e+04  2.910365e+00
max    1.262020e+05  1.262720e+05  1.261580e+05  1.262020e+05  5.853852e+03

## Weekday vs Weekend Volume T-Test
We test whether average trading activity differs between weekdays and weekends by comparing the log-transformed minute volumes with a Welch two-sample t-test and a simple boxplot.

In [64]:
weekday_mask = df1.index.dayofweek < 5
weekend_mask = ~weekday_mask

weekday_vol = np.log10(df1.loc[weekday_mask, "Volume"] + 1e-8)
weekend_vol = np.log10(df1.loc[weekend_mask, "Volume"] + 1e-8)

fig, ax = plt.subplots(figsize=(8, 5))
ax.boxplot([weekday_vol, weekend_vol], tick_labels=["Weekday", "Weekend"], widths=0.6)
ax.set_ylabel("log10(Volume + 1e-8)")
ax.set_title("Minute trading volume by market period")
plt.show()

t_stat, p_value = stats.ttest_ind(weekday_vol, weekend_vol, equal_var=False)
weekday_mean = weekday_vol.mean()
weekend_mean = weekend_vol.mean()
raw_ratio = df1.loc[weekday_mask, "Volume"].mean() / df1.loc[weekend_mask, "Volume"].mean()

print("Weekday mean log volume: ", weekday_mean,
      "\nWeekend mean log volume: ", weekend_mean,
      "\nT-value: ", t_stat,
      "\nP-Value: ", p_value) 
print(raw_ratio, "times more BTC per minute on weekdays.")

Weekday mean log volume:  -1.378667282497731 
Weekend mean log volume:  -2.029525121332803 
T-value:  246.08206494489278 
P-Value:  0.0
1.5748195751115495 times more BTC per minute on weekdays.


We compared minute bitcoin trading volumes between weekdays and weekends using a T-test on log10 because the raw minute volumes are very skewed. Weekdays show higher activity with mean log volume of -1.379 vs -2.030 which is around one and a half more raw volume per minute, with t=246 and p=0.0, so we reject the equal means. So we can conclude that the liquidity is higher on weekdays.

Next, moving to the second dataset, we make sure the 50 cryto-currency data is only relevant for the dates that we have data for Bitcoin. This is from 2012-01-01 and 2025-10-27.

In [65]:
df2 = pd.read_csv(path2, parse_dates=['Date'])
start = "2012-01-01" 
end = "2025-10-27"
mask = (df2['Date'] >= start) & (df2['Date'] <= end) 
df2 = df2.loc[mask] 
display(df2)

Currency_Name       Date    Price     Open     High      Low       Vol.  \
0              Aave 2018-01-30  0.15000  0.17000  0.17000  0.14000   530470.0   
1              Aave 2018-01-31  0.14000  0.15000  0.15000  0.13000   396050.0   
2              Aave 2018-02-01  0.11000  0.14000  0.14000  0.11000   987260.0   
3              Aave 2018-02-02  0.10000  0.11000  0.11000  0.08000  1810000.0   
4              Aave 2018-02-03  0.11000  0.10000  0.12000  0.09000  1200000.0   
...             ...        ...      ...      ...      ...      ...        ...   
53830           XPR 2015-01-26  0.01469  0.01700  0.01750  0.01469        0.0   
53831           XPR 2015-01-25  0.01700  0.01739  0.01739  0.01560        0.0   
53832           XPR 2015-01-24  0.01739  0.01600  0.01739  0.01500        0.0   
53833           XPR 2015-01-23  0.01600  0.01602  0.01700  0.01600        0.0   
53834           XPR 2015-01-22  0.01602  0.01523  0.01750  0.01523        0.0   

       Change %  
0         -7.95  
1        -11.10  
2        -17.46  
3         -8.32  
4          6.85  
...         ...  
53830    -13.59  
53831     -2.24  
53832      8.69  
53833     -0.12  
53834      5.19  

[53303 rows x 8 columns]

Check for null values

In [66]:
display(df2.isna().sum())

Currency_Name    0
Date             0
Price            0
Open             0
High             0
Low              0
Vol.             0
Change %         0
dtype: int64

No Null Values in dataset 2


Describing Dataset 2

In [67]:
display(df2.describe())

Date         Price          Open  \
count                          53303  53303.000000  53303.000000   
mean   2019-07-08 16:55:45.042492928    786.885588    785.204735   
min              2012-01-01 00:00:00      0.000000      0.000000   
25%              2018-07-30 00:00:00      0.297300      0.297010   
50%              2019-10-25 00:00:00      3.325700      3.319000   
75%              2020-11-02 00:00:00     67.405500     67.290000   
max              2021-08-24 00:00:00  63545.110000  63544.200000   
std                              NaN   4801.667173   4792.929378   

               High           Low          Vol.      Change %  
count  53303.000000  53303.000000  5.330300e+04  53303.000000  
mean     812.773439    754.798277  3.715733e+09      1.517813  
min        0.000000     -6.503000  0.000000e+00   -100.000000  
25%        0.310245      0.281905  1.371350e+05     -2.370000  
50%        3.487100      3.112300  2.180000e+06      0.000000  
75%       69.815000     64.493500  4.149500e+07      2.640000  
max    64831.500000  62067.500000  8.574631e+12  42751.340000  
std     4955.092001   4608.743419  8.824876e+10    189.242914

### Cumulative Return Analysis (2018-2021)

In [68]:
mask = (df2["Date"] >= "2018-01-01") & (df2["Date"] <= "2021-12-31")
filtered = df2.loc[mask].sort_values(["Currency_Name", "Date"]).copy()
filtered["Cumulative Return"] = (
    (1 + filtered["Change %"] / 100)
    .groupby(filtered["Currency_Name"])
    .cumprod() - 1
) * 100

fig = go.Figure()

for currency, subset in filtered.groupby("Currency_Name"):
    fig.add_trace(
        go.Scatter(
            x=subset["Date"],
            y=subset["Cumulative Return"],
            mode="lines",
            name=currency,
            line=dict(width=0.8),
            fill="tonexty",
            opacity=0.5
        )
    )

fig.add_hline(y=0, line=dict(color="black", width=1.5, dash="dot"))

fig.update_layout(
    title="Cumulative Growth of Major Cryptocurrencies (2018–2021)",
    title_x=0.5,
    xaxis_title="Date",
    yaxis_title="Total Return (%)",
    template="plotly_white",
    showlegend=True,
    margin=dict(l=60, r=40, t=90, b=60)
)

fig.update_xaxes(dtick="M6", tickformat="%b\n%Y", showgrid=True, gridcolor="lightgray")
fig.update_yaxes(ticksuffix="%", showgrid=True, gridcolor="lightgray", zeroline=True)

fig.show()


The filled cumulative return plot from 2018 to 2021 was chosen because it captures a complete market cycle in cryptocurrency history which includes the aftermath of the 2017 boom through the 2018 crash, 2020 pandemic shock, and the 2021 bull run thus, making it an ideal period to observe both downturns and recoveries. The visualization reveals that performance was highly uneven, with a few dominant coins such as Bitcoin and Ethereum showing strong, sustained growth while many others fluctuated near or below zero cumulative return. Moreover, I see that the wide spread and frequent crossings of the zero line highlight the market’s volatility and cyclical nature, where gains and losses occur in waves rather than stable trends. All in all, the time-series analysis indicates that long-term value creation in the crypto market was concentrated in a few leading assets, while most exhibited inconsistent growth, which underscored both the speculative and high-risk character of this market during the 2018–2021 period.
